<a href="https://colab.research.google.com/github/Ucchwas/Polysomnographic-Database-Respiration-and-ECG-features/blob/main/Polysomnographic_Database_(SLPDB).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wfdb
import wfdb
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import pandas as pd
import numpy as np
from scipy.stats import entropy
from scipy.signal import welch, find_peaks
!pip install pyentrp
from pyentrp import entropy as ent
!pip install tsfel
!pip install biosppy
import tsfel
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import RandomOverSampler
from tensorflow import keras
import tensorflow as tf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 26.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
astropy 5.3.4 requires numpy<2,>=1.21, but you have numpy 2.0.0 which is incompatible.
cudf-cu12 24.4.1 requires numpy<2.0a0,>=1.23, but you have numpy 2.0.0 which is incompatible.
cupy-cuda12x 12.2.0 requires numpy<1.27,>=1.20, but you have numpy 2.0.0 which is incompatible.
ibis-framework 8.0.0 requires numpy<2,>=1, but you have numpy 2.0.0 which is incompatible.
numba 0.58.1 requires numpy<1.27,>=1.22, but you have numpy 2.0.0 which is incompatible.
rmm-cu12 24.4.0 requires numpy<2.0a0,>=1.23, but you h

In [ ]:
record_names = ['slp02a', 'slp02b', 'slp03', 'slp04', 'slp14', 'slp16', 'slp32', 'slp37', 'slp41', 'slp45', 'slp48']
#'slp01a', 'slp01b', 'slp59', 'slp60', 'slp61', 'slp66', 'slp67x'

In [ ]:
import wfdb
import pandas as pd

# Initialize lists to store the signals and annotations
resp_signals = []
ecg_signals = []
resp_annotations_list = []

# Loop through each record name and read annotations
for name in record_names:
    annotations = wfdb.rdann(name, extension='st', pn_dir='slpdb')
    resp_annotations_list.append(annotations)

    record = wfdb.rdrecord(name, pn_dir='slpdb')
    resp_signals.append(record.p_signal[:, 3].tolist())  # Assuming the respiration signal is the 4th channel
    ecg_signals.append(record.p_signal[:, 0].tolist())   # Assuming the ECG signal is the 1st channel

# Save the signals and annotations to a CSV file
raw_data = {
    'record_name': record_names,
    'resp_signals': resp_signals,
    'ecg_signals': ecg_signals,
    'annotations_sample': [ann.sample.tolist() for ann in resp_annotations_list],
    'annotations_aux_note': [ann.aux_note for ann in resp_annotations_list]
}

raw_df = pd.DataFrame(raw_data)
raw_df.to_csv('SLPDB_Raw_Data_With_ECG.csv', index=False)

print("Raw data saved to 'SLPDB_Raw_Data_With_ECG.csv'")

Raw data saved to 'SLPDB_Raw_Data_With_ECG.csv'


In [ ]:
# from collections import Counter

# # Define the annotations of interest and their priority
# priority_annotations = {
#     "CA": "3",
#     "CAA": "3",
#     "OA": "2",
#     "X": "2",
#     "H": "1",
#     "HA": "1",
#     "W": "0"
# }

# # Initialize a counter for the sleep stages and apnea events
# stage_counts = Counter()

# # Sampling frequency and interval duration
# fs = 250
# interval_duration = 30

# # Function to get the highest priority annotation in a segment
# def get_priority_annotation(annotations, priority_annotations):
#     for ann in priority_annotations:
#         if ann in annotations:
#             return priority_annotations[ann]
#     return None

# # Loop through each annotation list and process the segments
# for resp_ann in resp_annotations_list:
#     # Create an array to mark the annotations for each sample
#     segment_annotations = [None] * (resp_ann.sample[-1] + 1)

#     # Mark the annotations in the array
#     for sample, symbol in zip(resp_ann.sample, resp_ann.aux_note):
#         annotations = symbol.strip().split()
#         for stage_key in annotations:
#             if stage_key in priority_annotations:
#                 segment_annotations[sample] = stage_key

#     # Process the segments of 30-second intervals
#     num_samples = len(segment_annotations)
#     num_intervals = num_samples // (fs * interval_duration)

#     for i in range(num_intervals):
#         start = i * fs * interval_duration
#         end = start + (fs * interval_duration)

#         # Get annotations in this segment
#         segment_ann = segment_annotations[start:end]

#         # Remove None values
#         segment_ann = [ann for ann in segment_ann if ann is not None]

#         if segment_ann:
#             # Get the highest priority annotation for this segment
#             priority_ann = get_priority_annotation(segment_ann, priority_annotations)
#             if priority_ann:
#                 stage_counts[priority_ann] += 1

# # Calculate the total number of intervals
# total_samples = sum(resp_ann.sample[-1] for resp_ann in resp_annotations_list)
# num_intervals = total_samples // (fs * interval_duration)

# # Display results
# print('Sleep Stage and Apnea Event Counts at 30-Second Intervals:')
# for stage, count in stage_counts.items():
#     print(f'{stage}: {count} occurrences in {num_intervals:.2f} intervals')

# print('\nPercentage of Each Stage:')
# for stage, count in stage_counts.items():
#     percentage = (count / num_intervals) * 100
#     print(f'{stage}: {percentage:.2f}% of the time')

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import find_peaks, welch, cwt, ricker
from scipy.stats import entropy, kurtosis, skew
from pywt import wavedec

# Read the raw data from the CSV file
raw_df = pd.read_csv('SLPDB_Raw_Data_With_ECG.csv')

# Convert string representations back to lists
raw_df['resp_signals'] = raw_df['resp_signals'].apply(eval)
raw_df['ecg_signals'] = raw_df['ecg_signals'].apply(eval)
raw_df['annotations_sample'] = raw_df['annotations_sample'].apply(eval)
raw_df['annotations_aux_note'] = raw_df['annotations_aux_note'].apply(eval)

# Debug: Print a few entries to verify data integrity
print(raw_df.head())

# Initialize empty list to store all features
extracted_features_list = []

# Define the annotations of interest and their priority
priority_annotations = {
    "CA": 3,
    "CAA": 3,
    "OA": 2,
    "X": 2,
    "H": 1,
    "HA": 1,
    "W": 0
}

def hjorth_parameters(segment):
    """ Calculate Hjorth parameters: Activity, Mobility, and Complexity. """
    activity = np.var(segment)
    diff = np.diff(segment)
    mobility = np.sqrt(np.var(diff) / activity)
    diff2 = np.diff(diff)
    complexity = np.sqrt(np.var(diff2) / np.var(diff)) / mobility
    return activity, mobility, complexity

def wavelet_features(segment, wavelet='db1'):
    """ Extract wavelet features from the signal. """
    coeffs = wavedec(segment, wavelet)
    features = []
    for coeff in coeffs:
        features.extend([np.mean(coeff), np.std(coeff), skew(coeff), kurtosis(coeff)])
    return features

def get_priority_annotation(annotations, priority_annotations):
    """ Get the highest priority annotation in a segment. """
    for ann in priority_annotations:
        if ann in annotations:
            return priority_annotations[ann]
    return None

  record_name                                       resp_signals  \
0      slp02a  [-0.03225806451612903, -0.016129032258064516, ...   
1      slp02b  [-0.5645161290322581, -0.5645161290322581, -0....   
2       slp03  [0.32304038004750596, 0.332541567695962, 0.327...   
3       slp04  [0.048154093097913325, 0.04735152487961477, 0....   
4       slp14  [0.051036682615629984, 0.04585326953748006, 0....   

                                         ecg_signals  \
0  [-0.008, -0.008, -0.012, -0.008, -0.0, -0.0, 0...   
1  [0.218, 0.218, 0.212, 0.214, 0.212, 0.2, 0.192...   
2  [0.028, 0.026, -0.006, 0.006, -0.008, -0.054, ...   
3  [0.416, 0.41, 0.406, 0.398, 0.4, 0.386, 0.37, ...   
4  [0.04, 0.046, 0.05, 0.056, 0.058, 0.058, 0.072...   

                                  annotations_sample  \
0  [1, 7500, 15000, 22500, 30000, 37500, 45000, 5...   
1  [1, 7500, 15000, 22500, 30000, 37500, 45000, 5...   
2  [1, 7500, 15000, 22500, 30000, 37500, 45000, 5...   
3  [1, 7500, 15000, 22500, 300

In [ ]:
# Loop through each row in the raw data
for index, row in raw_df.iterrows():
    record_name = row['record_name']
    resp_signals = np.array(row['resp_signals'])
    ecg_signals = np.array(row['ecg_signals'])
    annotations_sample = np.array(row['annotations_sample'], dtype=int)
    annotations_aux_note = row['annotations_aux_note']

    # Create an array to mark the annotations for each sample
    segment_annotations = [None] * (annotations_sample[-1] + 1)

    # Mark the annotations in the array
    for sample, symbol in zip(annotations_sample, annotations_aux_note):
        annotations = symbol.strip().split()
        for stage_key in annotations:
            if stage_key in priority_annotations:
                segment_annotations[sample] = stage_key

    # Process the segments of 30-second intervals
    num_samples = len(segment_annotations)
    num_intervals = num_samples // (250 * 30)  # Assuming the sample rate is 250 Hz and intervals are 30 seconds

    for i in range(num_intervals):
        start = i * 250 * 30
        end = start + (250 * 30)

        # Get annotations in this segment
        segment_ann = segment_annotations[start:end]

        # Remove None values
        segment_ann = [ann for ann in segment_ann if ann is not None]

        # Check if segment contains only expected labels
        if not segment_ann or any(ann not in priority_annotations for ann in segment_ann):
            continue  # Skip segments with unexpected annotations

        # Get the highest priority annotation for this segment
        priority_ann = get_priority_annotation(segment_ann, priority_annotations)
        if priority_ann is not None:
            # Extract features from the respiratory segment
            resp_segment = resp_signals[start:end]

            # Ensure the segment is non-zero and normalized
            if np.sum(resp_segment) == 0 or len(resp_segment) < 2:
                continue

            resp_segment = resp_segment - np.min(resp_segment)  # Shift to make all values positive
            if np.max(resp_segment) != 0:
                resp_segment = resp_segment / np.max(resp_segment)  # Normalize to max value 1

            # Time domain features for respiratory segment
            resp_mean = np.mean(resp_segment)
            resp_std = np.std(resp_segment)
            resp_skewness = skew(resp_segment)
            resp_area_abs = np.sum(np.abs(resp_segment))
            resp_kurt = kurtosis(resp_segment)
            resp_min_value = np.min(resp_segment)
            resp_max_value = np.max(resp_segment)
            resp_rms = np.sqrt(np.mean(resp_segment ** 2))
            resp_zcr = (np.diff(np.sign(resp_segment)) != 0).sum()
            resp_signal_energy = np.sum(resp_segment ** 2)
            resp_entropy_signal = entropy(resp_segment + 1e-10, base=2)

            # Hjorth parameters for respiratory segment
            resp_activity, resp_mobility, resp_complexity = hjorth_parameters(resp_segment)

            # Wavelet features for respiratory segment
            resp_wavelet_features = wavelet_features(resp_segment)

            # Peak-related features using find_peaks for respiratory segment
            resp_peaks, _ = find_peaks(resp_segment)
            if len(resp_peaks) > 0:
                resp_peak_heights = resp_segment[resp_peaks]
                resp_mean_peak_height = np.mean(resp_peak_heights)
                resp_std_peak_height = np.std(resp_peak_heights)
                resp_skewness_peak_height = skew(resp_peak_heights)
                resp_min_peak_height = np.min(resp_peak_heights)
                resp_max_peak_height = np.max(resp_peak_heights)
                resp_num_peaks = len(resp_peaks)
                if resp_num_peaks > 1:
                    resp_peak_distances = np.diff(resp_peaks)
                    resp_mean_inter_peak_distance = np.mean(resp_peak_distances)
                    resp_std_peak_distance = np.std(resp_peak_distances)
                    resp_skewness_inter_peak_distance = skew(resp_peak_distances)
                else:
                    resp_mean_inter_peak_distance = resp_std_peak_distance = resp_skewness_inter_peak_distance = 0
            else:
                resp_mean_peak_height = resp_std_peak_height = resp_skewness_peak_height = 0
                resp_min_peak_height = resp_max_peak_height = 0
                resp_num_peaks = 0
                resp_mean_inter_peak_distance = resp_std_peak_distance = resp_skewness_inter_peak_distance = 0
            resp_sum_peak_heights = np.sum(resp_peak_heights) if len(resp_peaks) > 0 else 0
            resp_peak_frequency = len(resp_segment) / np.sum(resp_segment > (np.max(resp_segment) / 2))

            # Frequency domain features for respiratory segment
            resp_fft_result = np.fft.fft(resp_segment)
            resp_magnitude_spectrum = np.abs(resp_fft_result[:len(resp_fft_result)//2])  # Take the positive frequencies
            resp_power_spectrum = resp_magnitude_spectrum ** 2
            resp_dominant_frequency = np.argmax(resp_power_spectrum)
            resp_total_power = np.sum(resp_power_spectrum)
            resp_mean_frequency = np.sum(resp_magnitude_spectrum * np.arange(len(resp_magnitude_spectrum))) / np.sum(resp_magnitude_spectrum)
            resp_central_frequency = np.argmax(resp_magnitude_spectrum[len(resp_magnitude_spectrum) // 2:]) + len(resp_magnitude_spectrum) // 2
            resp_low_frequency_index = 0
            resp_high_frequency_index = len(resp_power_spectrum) // 2
            resp_band_power = np.sum(resp_power_spectrum[resp_low_frequency_index:resp_high_frequency_index])
            resp_f, resp_pxx = welch(resp_segment, fs=250, nperseg=len(resp_segment)//2)
            resp_spectral_centroid = np.sum(resp_f * resp_pxx) / np.sum(resp_pxx)
            resp_spectral_spread = np.sqrt(np.sum(((resp_f - resp_spectral_centroid) ** 2) * resp_pxx) / np.sum(resp_pxx))
            resp_spectral_skewness = np.sum(((resp_f - resp_spectral_centroid) ** 3) * resp_pxx) / (resp_spectral_spread ** 3)
            resp_spectral_kurtosis = np.sum(((resp_f - resp_spectral_centroid) ** 4) * resp_pxx) / (resp_spectral_spread ** 4)

            # Slope-related features for respiratory segment
            resp_slopes = np.diff(resp_segment)
            resp_mean_slope = np.mean(resp_slopes)
            resp_std_slope = np.std(resp_slopes)

            # Statistical Features for respiratory segment
            resp_median = np.median(resp_segment)
            resp_percentile_25 = np.percentile(resp_segment, 25)
            resp_percentile_75 = np.percentile(resp_segment, 75)
            resp_data_range = np.max(resp_segment) - np.min(resp_segment)

            # Extract features from the ECG segment
            ecg_segment = ecg_signals[start:end]

            # Ensure the segment is non-zero and normalized
            if np.sum(ecg_segment) == 0 or len(ecg_segment) < 2:
                continue

            ecg_segment = ecg_segment - np.min(ecg_segment)  # Shift to make all values positive
            if np.max(ecg_segment) != 0:
                ecg_segment = ecg_segment / np.max(ecg_segment)  # Normalize to max value 1

            # Time domain features for ECG segment
            ecg_mean = np.mean(ecg_segment)
            ecg_std = np.std(ecg_segment)
            ecg_skewness = skew(ecg_segment)
            ecg_area_abs = np.sum(np.abs(ecg_segment))
            ecg_kurt = kurtosis(ecg_segment)
            ecg_min_value = np.min(ecg_segment)
            ecg_max_value = np.max(ecg_segment)
            ecg_rms = np.sqrt(np.mean(ecg_segment ** 2))
            ecg_zcr = (np.diff(np.sign(ecg_segment)) != 0).sum()
            ecg_signal_energy = np.sum(ecg_segment ** 2)
            ecg_entropy_signal = entropy(ecg_segment + 1e-10, base=2)

            # Hjorth parameters for ECG segment
            ecg_activity, ecg_mobility, ecg_complexity = hjorth_parameters(ecg_segment)

            # Wavelet features for ECG segment
            ecg_wavelet_features = wavelet_features(ecg_segment)

            # Peak-related features using find_peaks for ECG segment
            ecg_peaks, _ = find_peaks(ecg_segment)
            if len(ecg_peaks) > 0:
                ecg_peak_heights = ecg_segment[ecg_peaks]
                ecg_mean_peak_height = np.mean(ecg_peak_heights)
                ecg_std_peak_height = np.std(ecg_peak_heights)
                ecg_skewness_peak_height = skew(ecg_peak_heights)
                ecg_min_peak_height = np.min(ecg_peak_heights)
                ecg_max_peak_height = np.max(ecg_peak_heights)
                ecg_num_peaks = len(ecg_peaks)
                if ecg_num_peaks > 1:
                    ecg_peak_distances = np.diff(ecg_peaks)
                    ecg_mean_inter_peak_distance = np.mean(ecg_peak_distances)
                    ecg_std_peak_distance = np.std(ecg_peak_distances)
                    ecg_skewness_inter_peak_distance = skew(ecg_peak_distances)
                else:
                    ecg_mean_inter_peak_distance = ecg_std_peak_distance = ecg_skewness_inter_peak_distance = 0
            else:
                ecg_mean_peak_height = ecg_std_peak_height = ecg_skewness_peak_height = 0
                ecg_min_peak_height = ecg_max_peak_height = 0
                ecg_num_peaks = 0
                ecg_mean_inter_peak_distance = ecg_std_peak_distance = ecg_skewness_inter_peak_distance = 0
            ecg_sum_peak_heights = np.sum(ecg_peak_heights) if len(ecg_peaks) > 0 else 0
            ecg_peak_frequency = len(ecg_segment) / np.sum(ecg_segment > (np.max(ecg_segment) / 2))

            # Frequency domain features for ECG segment
            ecg_fft_result = np.fft.fft(ecg_segment)
            ecg_magnitude_spectrum = np.abs(ecg_fft_result[:len(ecg_fft_result)//2])  # Take the positive frequencies
            ecg_power_spectrum = ecg_magnitude_spectrum ** 2
            ecg_dominant_frequency = np.argmax(ecg_power_spectrum)
            ecg_total_power = np.sum(ecg_power_spectrum)
            ecg_mean_frequency = np.sum(ecg_magnitude_spectrum * np.arange(len(ecg_magnitude_spectrum))) / np.sum(ecg_magnitude_spectrum)
            ecg_central_frequency = np.argmax(ecg_magnitude_spectrum[len(ecg_magnitude_spectrum) // 2:]) + len(ecg_magnitude_spectrum) // 2
            ecg_low_frequency_index = 0
            ecg_high_frequency_index = len(ecg_power_spectrum) // 2
            ecg_band_power = np.sum(ecg_power_spectrum[ecg_low_frequency_index:ecg_high_frequency_index])
            ecg_f, ecg_pxx = welch(ecg_segment, fs=250, nperseg=len(ecg_segment)//2)
            ecg_spectral_centroid = np.sum(ecg_f * ecg_pxx) / np.sum(ecg_pxx)
            ecg_spectral_spread = np.sqrt(np.sum(((ecg_f - ecg_spectral_centroid) ** 2) * ecg_pxx) / np.sum(ecg_pxx))
            ecg_spectral_skewness = np.sum(((ecg_f - ecg_spectral_centroid) ** 3) * ecg_pxx) / (ecg_spectral_spread ** 3)
            ecg_spectral_kurtosis = np.sum(((ecg_f - ecg_spectral_centroid) ** 4) * ecg_pxx) / (ecg_spectral_spread ** 4)

            # Slope-related features for ECG segment
            ecg_slopes = np.diff(ecg_segment)
            ecg_mean_slope = np.mean(ecg_slopes)
            ecg_std_slope = np.std(ecg_slopes)

            # Statistical Features for ECG segment
            ecg_median = np.median(ecg_segment)
            ecg_percentile_25 = np.percentile(ecg_segment, 25)
            ecg_percentile_75 = np.percentile(ecg_segment, 75)
            ecg_data_range = np.max(ecg_segment) - np.min(ecg_segment)

            # Store features along with label
            features = [
                # Respiratory features
                resp_mean, resp_std, resp_skewness, resp_area_abs, resp_kurt, resp_min_value, resp_max_value,
                resp_rms, resp_zcr, resp_signal_energy, resp_entropy_signal,
                resp_mean_peak_height, resp_std_peak_height, resp_skewness_peak_height,
                resp_num_peaks, resp_mean_inter_peak_distance, resp_std_peak_distance, resp_skewness_inter_peak_distance,
                resp_sum_peak_heights, resp_peak_frequency, resp_max_peak_height, resp_min_peak_height,
                resp_dominant_frequency, resp_central_frequency, resp_band_power,
                resp_spectral_centroid, resp_spectral_spread, resp_spectral_skewness, resp_spectral_kurtosis,
                resp_mean_slope, resp_std_slope,
                resp_activity, resp_mobility, resp_complexity,
                resp_median, resp_percentile_25, resp_percentile_75, resp_data_range,
                *resp_wavelet_features,

                # ECG features
                ecg_mean, ecg_std, ecg_skewness, ecg_area_abs, ecg_kurt, ecg_min_value, ecg_max_value,
                ecg_rms, ecg_zcr, ecg_signal_energy, ecg_entropy_signal,
                ecg_mean_peak_height, ecg_std_peak_height, ecg_skewness_peak_height,
                ecg_num_peaks, ecg_mean_inter_peak_distance, ecg_std_peak_distance, ecg_skewness_inter_peak_distance,
                ecg_sum_peak_heights, ecg_peak_frequency, ecg_max_peak_height, ecg_min_peak_height,
                ecg_dominant_frequency, ecg_central_frequency, ecg_band_power,
                ecg_spectral_centroid, ecg_spectral_spread, ecg_spectral_skewness, ecg_spectral_kurtosis,
                ecg_mean_slope, ecg_std_slope,
                ecg_activity, ecg_mobility, ecg_complexity,
                ecg_median, ecg_percentile_25, ecg_percentile_75, ecg_data_range,
                *ecg_wavelet_features,

                # Label
                priority_ann
            ]

            extracted_features_list.append(features)

# Convert the list of extracted features into a DataFrame
df = pd.DataFrame(extracted_features_list)

# Fill NaN values with 0
df = df.fillna(0)

# Define column names
resp_time_domain_features = ['resp_mean', 'resp_std', 'resp_skewness', 'resp_area_abs', 'resp_kurt', 'resp_min_value', 'resp_max_value', 'resp_rms', 'resp_zcr', 'resp_signal_energy', 'resp_entropy_signal']
resp_peak_related_features = ['resp_mean_peak_height', 'resp_std_peak_height', 'resp_skewness_peak_height', 'resp_num_peaks', 'resp_mean_inter_peak_distance', 'resp_std_peak_distance', 'resp_skewness_inter_peak_distance', 'resp_sum_peak_heights', 'resp_peak_frequency', 'resp_max_peak_height', 'resp_min_peak_height']
resp_frequency_domain_features = ['resp_dominant_frequency', 'resp_central_frequency', 'resp_band_power', 'resp_spectral_centroid', 'resp_spectral_spread', 'resp_spectral_skewness', 'resp_spectral_kurtosis']
resp_slope_features = ['resp_mean_slope', 'resp_std_slope']
resp_hjorth_features = ['resp_activity', 'resp_mobility', 'resp_complexity']
resp_statistical_features = ['resp_median', 'resp_percentile_25', 'resp_percentile_75', 'resp_data_range']
resp_wavelet_features_names = [f'resp_wavelet_{i}' for i in range(4 * len(wavedec(resp_segment, 'db1')))]  # 4 features per wavelet coefficient set

ecg_time_domain_features = ['ecg_mean', 'ecg_std', 'ecg_skewness', 'ecg_area_abs', 'ecg_kurt', 'ecg_min_value', 'ecg_max_value', 'ecg_rms', 'ecg_zcr', 'ecg_signal_energy', 'ecg_entropy_signal']
ecg_peak_related_features = ['ecg_mean_peak_height', 'ecg_std_peak_height', 'ecg_skewness_peak_height', 'ecg_num_peaks', 'ecg_mean_inter_peak_distance', 'ecg_std_peak_distance', 'ecg_skewness_inter_peak_distance', 'ecg_sum_peak_heights', 'ecg_peak_frequency', 'ecg_max_peak_height', 'ecg_min_peak_height']
ecg_frequency_domain_features = ['ecg_dominant_frequency', 'ecg_central_frequency', 'ecg_band_power', 'ecg_spectral_centroid', 'ecg_spectral_spread', 'ecg_spectral_skewness', 'ecg_spectral_kurtosis']
ecg_slope_features = ['ecg_mean_slope', 'ecg_std_slope']
ecg_hjorth_features = ['ecg_activity', 'ecg_mobility', 'ecg_complexity']
ecg_statistical_features = ['ecg_median', 'ecg_percentile_25', 'ecg_percentile_75', 'ecg_data_range']
ecg_wavelet_features_names = [f'ecg_wavelet_{i}' for i in range(4 * len(wavedec(ecg_segment, 'db1')))]  # 4 features per wavelet coefficient set

label_column = ['label']

column_names = (resp_time_domain_features + resp_peak_related_features + resp_frequency_domain_features +
                resp_slope_features + resp_hjorth_features + resp_statistical_features + resp_wavelet_features_names +
                ecg_time_domain_features + ecg_peak_related_features + ecg_frequency_domain_features +
                ecg_slope_features + ecg_hjorth_features + ecg_statistical_features + ecg_wavelet_features_names +
                label_column)

# Rename the columns of the DataFrame
df.columns = column_names

# Save the DataFrame to a CSV file with labeled columns and excluding the index
df.to_csv('SLPDB_Resp_ECG_Features_Enhanced.csv', index=False)

# Print the shape of the DataFrame
print("Shape of the DataFrame:", df.shape)

Shape of the DataFrame: (3744, 181)


In [ ]:
# import numpy as np
# import pandas as pd
# from scipy.signal import find_peaks, welch
# from scipy.stats import entropy, kurtosis, skew

# # Read the raw data from the CSV file
# raw_df = pd.read_csv('SLPDB_Raw_Data.csv')

# # Convert string representations back to lists
# raw_df['resp_signals'] = raw_df['resp_signals'].apply(eval)
# raw_df['annotations_sample'] = raw_df['annotations_sample'].apply(eval)
# raw_df['annotations_aux_note'] = raw_df['annotations_aux_note'].apply(eval)

# # Debug: Print a few entries to verify data integrity
# print(raw_df.head())

# # Initialize empty list to store all features
# extracted_features_list = []

# # Define the annotations of interest and their priority
# priority_annotations = {
#     "CA": 3,
#     "CAA": 3,
#     "OA": 2,
#     "X": 2,
#     "H": 1,
#     "HA": 1,
#     "W": 0
# }

# def hjorth_parameters(segment):
#     """ Calculate Hjorth parameters: Activity, Mobility, and Complexity. """
#     activity = np.var(segment)
#     diff = np.diff(segment)
#     mobility = np.sqrt(np.var(diff) / activity)
#     diff2 = np.diff(diff)
#     complexity = np.sqrt(np.var(diff2) / np.var(diff)) / mobility
#     return activity, mobility, complexity

# # Function to get the highest priority annotation in a segment
# def get_priority_annotation(annotations, priority_annotations):
#     for ann in priority_annotations:
#         if ann in annotations:
#             return priority_annotations[ann]
#     return None

# # Loop through each row in the raw data
# for index, row in raw_df.iterrows():
#     record_name = row['record_name']
#     resp_signals = np.array(row['resp_signals'])
#     annotations_sample = np.array(row['annotations_sample'], dtype=int)
#     annotations_aux_note = row['annotations_aux_note']

#     # Create an array to mark the annotations for each sample
#     segment_annotations = [None] * (annotations_sample[-1] + 1)

#     # Mark the annotations in the array
#     for sample, symbol in zip(annotations_sample, annotations_aux_note):
#         annotations = symbol.strip().split()
#         for stage_key in annotations:
#             if stage_key in priority_annotations:
#                 segment_annotations[sample] = stage_key

#     # Process the segments of 30-second intervals
#     num_samples = len(segment_annotations)
#     num_intervals = num_samples // (250 * 30)  # Assuming the sample rate is 250 Hz and intervals are 30 seconds

#     for i in range(num_intervals):
#         start = i * 250 * 30
#         end = start + (250 * 30)

#         # Get annotations in this segment
#         segment_ann = segment_annotations[start:end]

#         # Remove None values
#         segment_ann = [ann for ann in segment_ann if ann is not None]

#         # Check if segment contains only expected labels
#         if not segment_ann or any(ann not in priority_annotations for ann in segment_ann):
#             continue  # Skip segments with unexpected annotations

#         # Get the highest priority annotation for this segment
#         priority_ann = get_priority_annotation(segment_ann, priority_annotations)
#         if priority_ann is not None:
#             # Extract features from the segment
#             segment = resp_signals[start:end]

#             # Ensure the segment is non-zero and normalized
#             if np.sum(segment) == 0 or len(segment) < 2:
#                 continue

#             segment = segment - np.min(segment)  # Shift to make all values positive
#             if np.max(segment) != 0:
#                 segment = segment / np.max(segment)  # Normalize to max value 1

#             # Time domain features
#             mean = np.mean(segment)
#             std = np.std(segment)
#             skewness = skew(segment)
#             area_abs = np.sum(np.abs(segment))
#             kurt = kurtosis(segment)
#             min_value = np.min(segment)
#             max_value = np.max(segment)
#             rms = np.sqrt(np.mean(segment ** 2))
#             zcr = (np.diff(np.sign(segment)) != 0).sum()
#             signal_energy = np.sum(segment ** 2)
#             entropy_signal = entropy(segment + 1e-10, base=2)

#             # Hjorth parameters
#             activity, mobility, complexity = hjorth_parameters(segment)

#             # Peak-related features using find_peaks
#             peaks, _ = find_peaks(segment)
#             if len(peaks) > 0:
#                 peak_heights = segment[peaks]
#                 mean_peak_height = np.mean(peak_heights)
#                 std_peak_height = np.std(peak_heights)
#                 skewness_peak_height = skew(peak_heights)
#                 min_peak_height = np.min(peak_heights)
#                 max_peak_height = np.max(peak_heights)
#                 num_peaks = len(peaks)
#                 if num_peaks > 1:
#                     peak_distances = np.diff(peaks)
#                     mean_inter_peak_distance = np.mean(peak_distances)
#                     std_peak_distance = np.std(peak_distances)
#                     skewness_inter_peak_distance = skew(peak_distances)
#                 else:
#                     mean_inter_peak_distance = std_peak_distance = skewness_inter_peak_distance = 0
#             else:
#                 mean_peak_height = std_peak_height = skewness_peak_height = 0
#                 min_peak_height = max_peak_height = 0
#                 num_peaks = 0
#                 mean_inter_peak_distance = std_peak_distance = skewness_inter_peak_distance = 0
#             sum_peak_heights = np.sum(peak_heights) if len(peaks) > 0 else 0
#             peak_frequency = len(segment) / np.sum(segment > (np.max(segment) / 2))

#             # Frequency domain features
#             fft_result = np.fft.fft(segment)
#             magnitude_spectrum = np.abs(fft_result[:len(fft_result)//2])  # Take the positive frequencies
#             power_spectrum = magnitude_spectrum ** 2
#             dominant_frequency = np.argmax(power_spectrum)
#             total_power = np.sum(power_spectrum)
#             mean_frequency = np.sum(magnitude_spectrum * np.arange(len(magnitude_spectrum))) / np.sum(magnitude_spectrum)
#             central_frequency = np.argmax(magnitude_spectrum[len(magnitude_spectrum) // 2:]) + len(magnitude_spectrum) // 2
#             low_frequency_index = 0
#             high_frequency_index = len(power_spectrum) // 2
#             band_power = np.sum(power_spectrum[low_frequency_index:high_frequency_index])
#             f, pxx = welch(segment, fs=250, nperseg=len(segment)//2)
#             spectral_centroid = np.sum(f * pxx) / np.sum(pxx)
#             spectral_spread = np.sqrt(np.sum(((f - spectral_centroid) ** 2) * pxx) / np.sum(pxx))
#             spectral_skewness = np.sum(((f - spectral_centroid) ** 3) * pxx) / (spectral_spread ** 3)
#             spectral_kurtosis = np.sum(((f - spectral_centroid) ** 4) * pxx) / (spectral_spread ** 4)

#             # Slope-related features
#             slopes = np.diff(segment)
#             mean_slope = np.mean(slopes)
#             std_slope = np.std(slopes)

#             # Statistical Features
#             median = np.median(segment)
#             percentile_25 = np.percentile(segment, 25)
#             percentile_75 = np.percentile(segment, 75)
#             data_range = np.max(segment) - np.min(segment)

#             # Store features along with label
#             features = [mean, std, skewness, area_abs, kurt, min_value, max_value,
#                         rms, zcr, signal_energy, entropy_signal,
#                         mean_peak_height, std_peak_height, skewness_peak_height,
#                         num_peaks, mean_inter_peak_distance, std_peak_distance, skewness_inter_peak_distance,
#                         sum_peak_heights, peak_frequency, max_peak_height, min_peak_height,
#                         dominant_frequency, central_frequency, band_power,
#                         spectral_centroid, spectral_spread, spectral_skewness, spectral_kurtosis,
#                         mean_slope, std_slope,
#                         activity, mobility, complexity,
#                         median, percentile_25, percentile_75, data_range,
#                         priority_ann]

#             extracted_features_list.append(features)

# # Convert the list of extracted features into a DataFrame
# df = pd.DataFrame(extracted_features_list)

# # Fill NaN values with 0
# df = df.fillna(0)

# # Define column names
# time_domain_features = ['mean', 'std', 'skewness', 'area_abs', 'kurt', 'min_value', 'max_value', 'rms', 'zcr', 'signal_energy', 'entropy_signal']
# peak_related_features = ['mean_peak_height', 'std_peak_height', 'skewness_peak_height', 'num_peaks', 'mean_inter_peak_distance', 'std_peak_distance', 'skewness_inter_peak_distance', 'sum_peak_heights', 'peak_frequency', 'max_peak_height', 'min_peak_height']
# frequency_domain_features = ['dominant_frequency', 'central_frequency', 'band_power', 'spectral_centroid', 'spectral_spread', 'spectral_skewness', 'spectral_kurtosis']
# slope_features = ['mean_slope', 'std_slope']
# hjorth_features = ['activity', 'mobility', 'complexity']
# statistical_features = ['median', 'percentile_25', 'percentile_75', 'data_range']
# label_column = ['label']

# column_names = time_domain_features + peak_related_features + frequency_domain_features + slope_features + hjorth_features + statistical_features + label_column

# # Rename the columns of the DataFrame
# df.columns = column_names

# # Save the DataFrame to a CSV file with labeled columns and excluding the index
# df.to_csv('SLPDB_Resp_Features_Enhanced.csv', index=False)

# # Print the shape of the DataFrame
# print("Shape of the DataFrame:", df.shape)

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the dataset
df = pd.read_csv('SLPDB_Resp_ECG_Features_Enhanced.csv')

# Convert all feature columns to numeric, handling errors by coercing to NaN
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with NaN values if any
df.dropna(inplace=True)

# Check the shape of the dataframe after dropping NaN values
print("Shape after dropping NaNs:", df.shape)

# Separate features and labels
X = df.drop(columns=['label'])
y = df['label']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA to preserve 95% of the variance
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

# Get the number of components selected
n_components_selected = pca.n_components_

# Get the original feature names that contribute most to each principal component
pca_components = pca.components_
top_features_names = []
for i in range(n_components_selected):
    component = pca_components[i]
    top_feature_idx = np.argmax(np.abs(component))
    top_feature_name = X.columns[top_feature_idx]
    top_features_names.append(top_feature_name)

# Create a new DataFrame with the selected top features
top_features_df = pd.DataFrame(X_pca, columns=top_features_names)

# Add the label column
top_features_df['label'] = y.values

# Sort the DataFrame by the label column
top_features_df_sorted = top_features_df.sort_values(by='label')

# Save the new DataFrame to a CSV file
top_features_df_sorted.to_csv(f'SLPDB_Top_{n_components_selected}_Features_PCA.csv', index=False)

print(f"Top {n_components_selected} features saved to 'SLPDB_Top_{n_components_selected}_Features_PCA.csv'")

Shape after dropping NaNs: (3744, 181)
Top 79 features saved to 'SLPDB_Top_79_Features_PCA.csv'


In [ ]:
from sklearn.model_selection import train_test_split
# Load the DataFrame containing the selected features and labels
df_selected_features = pd.read_csv('SLPDB_Top_79_Features_PCA.csv')

# Separate features and labels
X = df_selected_features.drop(columns=['label'])
y = df_selected_features['label']

# Split the dataset randomly
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert scaled features back to DataFrame
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

# Create DataFrames for training and testing sets
df_train = pd.concat([X_train_scaled_df, y_train.reset_index(drop=True)], axis=1)
df_test = pd.concat([X_test_scaled_df, y_test.reset_index(drop=True)], axis=1)

# Save the training and testing DataFrames to CSV files
df_train.to_csv('features_train.csv', index=False)
df_test.to_csv('features_test.csv', index=False)